In [66]:
# importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix


In [142]:
data_X = pd.read_csv('X_train.csv')
data_y = pd.read_csv('y_train.csv')
print(data_X.shape, data_y.shape)

(487680, 13) (3810, 3)


In [233]:
display(data_X.head())
# display(data_X.tail())

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,resultant_angular_velocity,resultant_linear_acceleration,euler_t1,euler_t2,euler_t3
0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,0.109076,10.005392,2.843272,-0.024665,1.396664
1,0,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128,0.074240,9.538638,2.843199,-0.024660,1.396646
2,0,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,0.030428,8.874697,2.843224,-0.024726,1.396682
3,0,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960,0.025082,10.164638,2.843186,-0.024767,1.396721
4,0,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410,0.010603,10.556132,2.843195,-0.024782,1.396693


,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,resultant_angular_velocity,resultant_linear_acceleration,euler_t1,euler_t2,euler_t3
487675,3809,0.62871,-0.76878,-0.084391,0.081093,0.003167,0.093760,-0.14274,3.27180,2.0115,-9.0063,0.170809,9.791029,2.907700,-0.018570,-1.768404
487676,3809,0.62884,-0.76868,-0.084365,0.081099,0.014994,0.032637,-0.13238,4.42750,3.0696,-8.1257,0.137166,9.749472,2.907732,-0.018574,-1.768079
487677,3809,0.62891,-0.76861,-0.084345,0.081178,-0.031184,-0.003961,-0.13894,2.70480,4.2622,-8.1443,0.142452,9.581853,2.907657,-0.018698,-1.767856
487678,3809,0.62903,-0.76850,-0.084414,0.081231,-0.069153,0.013229,-0.13021,2.54100,4.7130,-9.4435,0.148026,10.855816,2.907480,-0.018654,-1.767534
487679,3809,0.62915,-0.76839,-0.084441,0.081284,-0.042769,0.034049,-0.12580,0.82391,4.2751,-10.4980,0.137165,11.365004,2.907365,-0.018664,-1.767200


In [144]:
# data_X.info()

**Observation:** There are no null values, and 487680 records are there

In [145]:
# data_X.describe()

In [146]:
# print(data_y.shape)
# display(data_y.head())
# # display(data_y.tail())

# Feature Engineering

In [148]:
def resultant(x,y,z,w=0):
    return (x**2 + y**2 + z**2 + w**2)**0.5

def quaternion_to_euler_angle(q0, q1, q2, q3):
    a = 2*(q0*q1 + q2*q3)
    b = 1 - 2*(q1*q1 + q2*q2)
    t1 = np.arctan2(a,b)
    
    t2 = 2*(q0*q2 - q3*q1)
    
    c = 2*(q0*q3 + q1*q2)
    d = 1 - 2*(q2*q2 + q3*q3)
    t3 = np.arctan2(c,d)
    return pd.concat((t1, t2, t3), axis=1)

def feature_engg(data_X):
    data_X.drop(['row_id', 'measurement_number'], axis=1, inplace=True)
    
    data_X.fillna(0,inplace=True)
    data_X.replace(-np.inf,0,inplace=True)
    data_X.replace(np.inf,0,inplace=True)
    
    data_X["resultant_angular_velocity"] = resultant(data_X["angular_velocity_X"],data_X["angular_velocity_Y"],data_X["angular_velocity_Z"])
    data_X["resultant_linear_acceleration"] = resultant(data_X["linear_acceleration_X"],data_X["linear_acceleration_Y"],data_X["linear_acceleration_Z"])
#     data_X["resultant_orientation"] = resultant(data_X["orientation_W"],data_X["orientation_X"],data_X["orientation_Y"],data_X["orientation_Z"])
    
    
    data_X[["euler_t1", "euler_t2", "euler_t3"]] = quaternion_to_euler_angle(data_X['orientation_W'],data_X['orientation_X'],data_X['orientation_Y'],data_X['orientation_Z'])

    new_df = pd.DataFrame()

    funct1 = {'min':'min()','max':'max()','mean':'mean()','std':'std()', 'median':'median()'}
    for op in funct1:

        df = eval('data_X.groupby(["series_id"]).{}'.format(funct1[op]))
        df.columns = df.columns.map(lambda x: x+'_'+op)
        new_df = pd.concat([new_df, df], axis=1)
    
# data_X.groupby('series_id').apply(lambda c: c.abs().sum())    
#     -----------------------------------------------------------------    
#     funct2 = {'abs_max':['abs()','max()'], 'abs_min':['abs()','min()']}
#     for op in funct2:

#         df = eval('data_X.{}.groupby(["series_id"]).{}'.format(funct2[op][0], funct2[op][1]))
#         df.columns = df.columns.map(lambda x: x+'_'+op)
#         new_df = pd.concat([new_df, df], axis=1)
    
        
    return new_df

In [149]:
new_df = feature_engg(data_X)

print(new_df.shape)
new_df.head()

(3810, 75)


,orientation_X_min,orientation_Y_min,orientation_Z_min,orientation_W_min,angular_velocity_X_min,angular_velocity_Y_min,angular_velocity_Z_min,linear_acceleration_X_min,linear_acceleration_Y_min,linear_acceleration_Z_min,...,angular_velocity_Y_median,angular_velocity_Z_median,linear_acceleration_X_median,linear_acceleration_Y_median,linear_acceleration_Z_median,resultant_angular_velocity_median,resultant_linear_acceleration_median,euler_t1_median,euler_t2_median,euler_t3_median
series_id,,,,,,,,,,,,,,,,,,,,,
0,-0.75953,-0.63456,-0.106140,-0.107050,-0.16041,-0.079404,-0.030181,-1.8644,0.075417,-12.512,...,-0.004037,0.006842,0.231665,3.40755,-9.42995,0.052661,10.077860,2.841582,-0.025047,1.396673
1,-0.95896,0.24074,0.030504,-0.148090,-0.25480,-0.134330,-0.121610,-3.1934,-2.149200,-16.928,...,-0.006330,0.003538,0.003571,2.75010,-9.41380,0.101244,10.069950,2.840183,-0.010446,-0.492860
2,-0.51434,-0.84779,-0.130300,-0.071535,-0.15271,-0.107810,-0.015697,-2.5930,-1.254000,-12.499,...,-0.010880,0.028323,0.174515,3.03375,-9.37440,0.067325,10.082289,2.845432,-0.012183,2.055114
3,-0.93968,0.30943,0.037922,-0.144370,-0.40152,-0.168150,-0.073414,-3.7934,-5.825100,-19.845,...,0.000518,0.005856,0.317205,3.00885,-9.16170,0.127105,10.925683,2.845451,-0.015080,-0.635690
4,-0.89689,0.41646,0.058247,-0.137320,-0.10407,0.008231,-0.213940,-1.2696,0.342070,-10.975,...,0.047561,-0.146670,-0.054043,3.13565,-9.33280,0.159897,10.005620,2.842437,-0.009802,-0.895723


In [150]:
data_y.drop(['group_id'], axis=1, inplace=True)
data_y.set_index('series_id', inplace=True)

data_y.head()

,surface
series_id,
0,fine_concrete
1,concrete
2,concrete
3,concrete
4,soft_tiles


In [151]:
data_y['surface'].values

array(['fine_concrete', 'concrete', 'concrete', ..., 'fine_concrete',
       'tiled', 'soft_pvc'], dtype=object)

In [152]:
le = LabelEncoder()
data_y['surface']= le.fit_transform(data_y['surface'])

In [153]:
X_train, X_test, y_train, y_test = train_test_split(new_df, data_y, test_size=0.3, stratify=data_y['surface'].values, random_state=37)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2667, 75) (1143, 75) (2667, 1) (1143, 1)


# Model

### Basic Model (GBDT)

In [131]:
xgb_clf = XGBClassifier(n_estimators=700, use_label_encoder=False)

xgb_clf.fit(X_train, y_train, eval_metric='mlogloss', )

xgb_clf.

predict_y = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test, predict_y)
print('accuracy: ', accuracy)

accuracy:  0.9230096237970253


In [108]:
# features = X_train.columns
# importances = xgb_clf.feature_importances_
# indices = (np.argsort(importances))

# hm = 30
# best_features = features[indices[-hm:][::-1]]
# best_importances = importances[indices[-hm:][::-1]]

# plt.figure(figsize=(7, hm//3.5))
# plt.title('Feature Importances')

# sns.barplot(x=best_importances, y=best_features);

### Advance Model

In [65]:
x_cfl = XGBClassifier(use_label_encoder=False)
skf = StratifiedKFold(n_splits=3)
prams={
#     'learning_rate':[0.01,0.03,0.05,0.1,0.15,0.2],
     'n_estimators':[100,200,500,700,900],
#      'max_depth':[3,5,10],
#     'colsample_bytree':[0.1,0.3,0.5,1],
#     'subsample':[0.1,0.3,0.5,1]
}
random_cfl1 = RandomizedSearchCV(x_cfl, param_distributions=prams,scoring='accuracy', verbose=10, n_jobs=-1, cv=skf)
random_cfl1.fit(X_train,y_train)

print(random_cfl1.best_params_)

print('Test accuracy: ', random_cfl1.score(X_test, y_test))

C:\Users\Nehal\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Nehal\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  3.5min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  4.2min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.9min finished


[04:59:01] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'n_estimators': 100}
Test accuracy:  0.9133858267716536


In [190]:
random_cfl1.cv_results_['mean_test_score']

array([0.89632546, 0.89271654, 0.89238845, 0.89206037, 0.89206037])

In [198]:
random_cfl1.cv_results_['mean_train_score']

array([1., 1., 1., 1., 1.])

# DT

In [155]:
from sklearn.tree import DecisionTreeClassifier

params = { 'max_depth':[1, 5, 10, 50], 'min_samples_split':[5, 10, 100, 500] }

clf = DecisionTreeClassifier(class_weight='balanced') # since data is imbalanced so "class_weight='balanced'" is used here.
skf = StratifiedKFold()
gs = GridSearchCV(clf, param_grid=params, scoring='accuracy', n_jobs=-1, return_train_score=True, cv=skf)
gs.fit(X_train, y_train)

C:\Users\Nehal\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:624: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
C:\Users\Nehal\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [1, 5, 10, 50], 'min_samples_split': [5, 10, 100, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [156]:
gs.score(X_test, y_test)

0.7751531058617673

# kaggle test

In [133]:
kagg_test = pd.read_csv('X_test.csv')
print(kagg_test.shape)
kagg_test.head()

(488448, 13)


,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,0.91208,-0.38193,-0.050618,0.14028,-0.060205,0.071286,-0.18787,0.29492,2.8027,-9.6816
1,0_1,0,1,0.91220,-0.38165,-0.050573,0.14028,-0.033486,0.060210,-0.18206,0.14944,2.5408,-9.8521
2,0_2,0,2,0.91228,-0.38143,-0.050586,0.14032,-0.029686,0.029476,-0.18441,-0.49741,2.5853,-9.3835
3,0_3,0,3,0.91237,-0.38121,-0.050588,0.14035,-0.024217,0.037788,-0.18783,-0.32376,2.9966,-8.7415
4,0_4,0,4,0.91247,-0.38096,-0.050546,0.14042,-0.038047,0.083405,-0.20170,-0.70103,2.6498,-8.8432


In [134]:
kagg_test = feature_engg(kagg_test)
print(kagg_test.shape)

(3816, 75)


In [199]:
# kagg_test.head(10)

In [157]:
kagg_y_pred = gs.predict(kagg_test)

In [158]:
kagg_y_pred

array([5, 8, 2, ..., 1, 6, 5])

In [160]:
kagg_y_pred_df = pd.DataFrame(list(range(len(kagg_y_pred))), columns=['series_id'])
kagg_y_pred_df['surface']=le.inverse_transform(kagg_y_pred)
print(kagg_y_pred_df.shape)
kagg_y_pred_df.head()

(3816, 2)


,series_id,surface
0,0,soft_pvc
1,1,wood
2,2,fine_concrete
3,3,wood
4,4,concrete


In [202]:
# kagg_y_pred_df.to_csv('robo3.csv', index=False)

In [201]:
# a = pd.read_csv('submission.csv')
# b = pd.read_csv('robo3.csv')

# k=0
# for i in zip(a['surface'].values, b['surface'].values):
#     if i[0]!=i[1]:
#         k+=1
# print(k)

In [46]:
data_X.groupby('series_id').apply(lambda c: c.abs().sum())

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,resultant_angular_velocity,resultant_linear_acceleration,euler_t1,euler_t2,euler_t3
series_id,,,,,,,,,,,,,,,,
0,0.0,97.109220,81.152980,13.500690,13.628110,5.279737,3.380616,1.919517,97.911042,381.976947,1193.010000,7.381550,1272.278555,363.741973,3.204397,178.692530
1,128.0,122.701620,30.958970,4.051239,18.800070,9.218290,4.539781,7.536752,101.993821,367.803191,1201.779100,14.309121,1290.552849,363.536528,1.327150,63.069540
2,256.0,65.543340,108.309880,16.559440,9.098478,6.271455,3.594032,3.809751,78.563381,377.736515,1202.660200,9.099457,1284.574841,364.227651,1.560917,263.042744
3,384.0,120.213640,39.697940,4.986241,18.216810,18.078451,4.462933,3.359077,150.109006,530.030322,1209.749000,19.974420,1393.676007,364.259493,1.933651,81.363929
4,512.0,114.086470,54.802460,7.687225,17.466930,4.905417,5.901899,18.225288,44.789454,381.631730,1196.798500,20.206798,1266.280289,363.832589,1.253531,114.450875
5,640.0,59.483200,111.679510,16.289830,10.395072,10.272724,5.587323,5.335916,161.324592,381.275579,1188.511000,14.361850,1294.184122,363.440111,2.999122,276.379421
6,768.0,51.501580,115.602110,17.989310,6.636674,2.840134,2.344428,6.522462,48.788061,337.990400,1211.522500,7.915196,1262.458433,363.708019,2.488775,295.207713
7,896.0,118.516340,44.500050,5.961730,17.945810,12.979732,6.261537,6.140844,206.207040,381.480991,1194.488230,17.153833,1325.547855,364.190969,1.437898,91.737445
8,1024.0,1.541209,126.582040,18.893020,1.234443,1.687363,2.928888,7.242356,50.751040,315.599800,1217.706100,8.551091,1259.978175,364.167878,1.986715,398.710340


In [200]:
# dff = pd.DataFrame([[2,-1],[3, -7],[7,3],[-9, 11]])
# display(dff)
# dff.abs().max()

In [130]:
# 

### references

In [ ]:
# http://www.chrobotics.com/library/understanding-quaternions